In [1]:
import requests
import re
import os
from dotenv import load_dotenv
import pandas as pd
import io
import time
from datetime import datetime, timedelta


In [2]:
flag = 0
contador = 1
#&minmagnitude=4
df_all = pd.DataFrame()
for i in range(2020):
    if i >= 2000:
        for sem in range(2):
            if sem == 0:
                url = f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime={i}-01-01&endtime={i}-07-01&minmagnitude=2'
                # print(url,'\n')
            else:
                url = f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime={i}-07-01&endtime={i+1}-01-01&minmagnitude=2'
                # print(url)
            respons = requests.get(url)
            if respons.status_code == 200:
                cont = respons.content
                df = pd.read_csv(io.StringIO(cont.decode('utf-8')))
                if flag > 0:
                    df_all = df_all.append(df, ignore_index=True)
                else:
                    df_all = df
            else:
                print(f'No hay respuesta del servidor')
                print(url)
                break
            time.sleep(100)
            flag = 1
        # break

df_all.shape

No hay respuesta del servidor
https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2004-07-01&endtime=2005-01-01&minmagnitude=2
No hay respuesta del servidor
https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2005-01-01&endtime=2005-07-01&minmagnitude=2
No hay respuesta del servidor
https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2006-01-01&endtime=2006-07-01&minmagnitude=2
No hay respuesta del servidor
https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2008-01-01&endtime=2008-07-01&minmagnitude=2
No hay respuesta del servidor
https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2010-01-01&endtime=2010-07-01&minmagnitude=2
No hay respuesta del servidor
https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2018-01-01&endtime=2018-07-01&minmagnitude=2
No hay respuesta del servidor
https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2019-07-01&endtime=2020-01-01&

(454140, 22)

In [35]:
# 
# 
# 
# 
# 
# 
# url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2019-10-01&endtime=2020-01-01&minmagnitude=2'
# respons = requests.get(url)
# if respons.status_code == 200:
#     cont = respons.content
#     df = pd.read_csv(io.StringIO(cont.decode('utf-8')))
#     df_all = df_all.append(df, ignore_index=True)
# else:
#     print(f'No hay respuesta del servidor')
#     print(url)

In [36]:
df_all.shape

(617361, 22)

In [51]:
df_states_usa = pd.read_csv('../Output/usa_country.csv')

In [52]:
# Función para obtener la fecha 
def extraeFecha(cadena):
    fechaTiempo = cadena.split('T')
    return fechaTiempo[0]

# extraeFecha('2003-12-30T23:52:08.630Z')

In [53]:
#Se crea el campo fecha
df_all['date'] = pd.to_datetime(df_all['time'].apply(extraeFecha), format="%Y-%m-%d")

KeyError: 'time'

In [40]:
# Función para extraer el país del campo PLACE
def extract_country(place):
    if place == '':
        country = 'No_Country'
    else:
        place_split = place.split(',')
        if len(place_split) > 1:
            country = place_split[1]
        else:
            country = 'No_Country'
        
    return country.strip().upper()

In [41]:
# Se crea el campo PAIS  obteniendo el mismo del campo PLACE
df_all['place'].fillna('',inplace = True)
df_all['Country'] = df_all['place'].apply(extract_country)

In [54]:
# Se cambian los nombres de los estados por USA
for e in df_states_usa.index:
    state = df_states_usa['State'][e]
    df_all.loc[(df_all['Country'] == state.upper()), 'Country'] = 'USA'

In [ ]:
# Leer archivos
# df_all = pd.read_csv('..\Output\earthquaqkes_all.csv')
# df_mexico = pd.read_csv('..\Output\earthquaqkes_all_mexico.csv')

In [43]:
# Renombrar columnas
columns_array = {'mag':'Magnitud','depth':'Depth','country':'Country','date':'Date','latitude':'Latitude','longitude':'Longitude'}
df_all.rename(columns=columns_array, inplace = True)

In [44]:
# Convertir a mayúsculas nombres de paises
df_all['Country'] = df_all['Country'].apply(lambda x: x.upper())

In [45]:
# Se genera la columna de año
df_all['Year'] = pd.to_datetime(df_all['Date'], format="%Y-%m-%d").dt.year

In [ ]:
df_all.head()

In [46]:
# filtra solo las columnas que se requieren para el modelo 
df_all = df_all[df_all['type'] == 'earthquake'][['Latitude','Longitude','Depth','Magnitud','Date','Country','Year']]
# 'latitude','longitude','depth','mag','date','country'
# Se deben agregar con el otro proceso Region, Density y IDH
# Estas columnas se agregan con el procedimiento de modelado Death_Cat, Injuries_Cat, Damage_Cat

In [47]:
df_all.head()

,Latitude,Longitude,Depth,Magnitud,Date,Country,Year
0,44.269,146.265,72.9,4.9,2000-06-30,RUSSIA,2000
1,-39.230,177.470,36.0,3.4,2000-06-30,NEW ZEALAND,2000
2,50.666,6.124,12.6,2.6,2000-06-30,BELGIUM,2000
4,37.399,141.503,42.1,5.4,2000-06-30,JAPAN,2000
5,-3.272,138.539,33.0,3.7,2000-06-30,INDONESIA,2000


In [48]:
#Obtiene los registros de Mexico
df_mexico = df_all[df_all['Country'] == 'MEXICO']
df_mexico.head()

,Latitude,Longitude,Depth,Magnitud,Date,Country,Year
23,17.928,-102.953,30.6,4.4,2000-06-30,MEXICO,2000
25,17.014,-100.317,74.0,3.9,2000-06-30,MEXICO,2000
214,17.921,-100.476,78.4,3.7,2000-06-28,MEXICO,2000
369,20.723,-99.483,14.0,3.4,2000-06-26,MEXICO,2000
371,16.231,-98.051,8.0,3.8,2000-06-26,MEXICO,2000


In [ ]:
df_mexico['Date'].max()

In [55]:
# Eliminación registros duplicados
df_all.drop_duplicates()
# Se guarda la informacion de todos los eventos desde 2000 hasta 2019
df_all.to_csv('..\Output\earthquaqkes_all_clean.csv', index=False)
df_mexico.to_csv('..\Output\earthquaqkes_all_mexico_clean.csv', index=False)